In [7]:
import requests, re
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
r=requests.get("https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
c=r.content

soup=BeautifulSoup(c,"html.parser")

Necesito conocer la estructura de la web, por lo que lo adecuado es inspeccionarla y a partir de ahí seguir extrayendo datos

In [9]:
all = soup.find_all("div", {"class": "propertyRow"})
print(len(all)) # Debe ser 10, que son los resultados que hay en una página

all[0].find_all

10


<bound method Tag.find_all of <div class="propertyRow" id="propertyRowREN021201395" onclick="Track.doEvent('Hybrid Mapping', 'Property Center Lane', 'Select a property with brand REN to view details'); document.location.href='/property/0-gateway-rock-springs-wy-82901-REN021201395';">
<div class="CenterLaneCardBg CardWrapper propertyCard" id="propertyREN021201395">
<div class="CenterLaneCard propertyCard">
<div class="CardThumb">
<div class="landscapeThumbContainer">
<a href="http://web.archive.org/web/20160127020422/http://www.century21.com/property/0-gateway-rock-springs-wy-82901-REN021201395"><img class="lazyLoad" rel="http://www.century21.com/listhub/thumb/130x90/photos.listhub.net/FCBRWY/20156419/1?lm=20151103T230938" src="./LCWYROCKSPRINGS1_files/1"/></a>
</div>
</div>
<h4 class="propPrice">


            $725,000



      <span class="IconPropertyFavorite16"></span>
</h4>
<div class="CardDetails">
<div class="primaryDetails">
<span class="propAddressCollapse" title="0 Gateway">0 

In [10]:
all[0].find_all("h4", {"class": "propPrice"})

[<h4 class="propPrice">
 
 
             $725,000
 
 
 
       <span class="IconPropertyFavorite16"></span>
 </h4>]

In [11]:
all[0].find("h4", {"class": "propPrice"}).text

'\n\n\n            $725,000\n\n\n\n      \n'

In [12]:
all[0].find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", "") 
# Reemplazo los \n y los espacios en blanco para que quede solo el precio

'$725,000'

In [13]:
l = []

for item in all:
    d={} 
    
    d["Address"] = item.find_all("span", {"class": "propAddressCollapse"})[0].text
    d["Locality"] = item.find_all("span", {"class": "propAddressCollapse"})[1].text
    d["Price"] = item.find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", "")

    try:  # Si no hay info sobre las camas no dará error
        d["Beds"] = item.find("span", {"class": "infoBed"}).text
        #print(item.find("span", {"class": "infoBed"}).find("b").text)  # Otra forma de hacerlo
    except:
        d["Beds"] = None # No hay info sobre las camas, por lo que imprimo None


    try:
        d["Area"] = item.find("span", {"class": "infoSqFt"}).find("b").text
    except:
        d["Area"] = None


    try:
        d["Full Baths"] = item.find("span", {"class": "infoValueFullBath"}).find("b").text
    except:
        d["Full Baths"] = None


    try:
        d["Half Baths"] = item.find("span", {"class": "infoValueHalfBath"}).find("b").text
    except:
        d["Half Baths"] = None
    
    for column_group in item.find_all("div", {"class": "columnGroup"}):
        for feature_group, feature_name in zip(column_group.find_all("span", {"class": "featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
            if "Lot Size" in feature_group.text:
                d["Lot Size"] = feature_name.text


    l.append(d)

In [14]:
print(len(l))
print(l)

10
[{'Address': '0 Gateway', 'Locality': 'Rock Springs, WY 82901', 'Price': '$725,000', 'Beds': None, 'Area': None, 'Full Baths': None, 'Half Baths': None}, {'Address': '1003 Winchester Blvd.', 'Locality': 'Rock Springs, WY 82901', 'Price': '$452,900', 'Beds': '4 Beds', 'Area': None, 'Full Baths': '4', 'Half Baths': None, 'Lot Size': '0.21 Acres'}, {'Address': '600 Talladega', 'Locality': 'Rock Springs, WY 82901', 'Price': '$396,900', 'Beds': '5 Beds', 'Area': '3,154', 'Full Baths': '3', 'Half Baths': None}, {'Address': '3239 Spearhead Way', 'Locality': 'Rock Springs, WY 82901', 'Price': '$389,900', 'Beds': '4 Beds', 'Area': '3,076', 'Full Baths': '3', 'Half Baths': '1', 'Lot Size': 'Under 1/2 Acre, '}, {'Address': '522 Emerald Street', 'Locality': 'Rock Springs, WY 82901', 'Price': '$254,000', 'Beds': '3 Beds', 'Area': '1,172', 'Full Baths': '3', 'Half Baths': None, 'Lot Size': 'Under 1/2 Acre, '}, {'Address': "1302 Veteran's Drive", 'Locality': 'Rock Springs, WY 82901', 'Price': '$25

In [15]:
df = pd.DataFrame(l)
df

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4 Beds,None,4,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5 Beds,"3,154",3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4 Beds,"3,076",3,1,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3 Beds,"1,172",3,None,"Under 1/2 Acre,"
5,1302 Veteran's Drive,"Rock Springs, WY 82901","$252,900",4 Beds,"1,932",2,None,0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000",4 Beds,"1,676",3,None,"Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000",3 Beds,"1,344",2,None,"Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900",3 Beds,"1,920",2,None,"Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900",3 Beds,"1,664",2,None,2.02 Acres


In [16]:
df.to_csv("Output.csv")

In [17]:
# Si en la web cambiamos de página la url resultante es la siguiente en la que con cada cambio de página se incrementa en 10.
# Página 1 -> https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
# Página 2 -> https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html

base_url = "https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="

for page in range(0,30,10):
    print(base_url + str(page) + ".html")
    r = requests.get(base_url + str(page) + ".html")
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    all = soup.find_all("div", {"class": "propertyRow"})

    for item in all:
        d={} 
        
        d["Address"] = item.find_all("span", {"class": "propAddressCollapse"})[0].text
        d["Locality"] = item.find_all("span", {"class": "propAddressCollapse"})[1].text
        d["Price"] = item.find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", "")

        try:  # Si no hay info sobre las camas no dará error
            d["Beds"] = item.find("span", {"class": "infoBed"}).text
            #print(item.find("span", {"class": "infoBed"}).find("b").text)  # Otra forma de hacerlo
        except:
            d["Beds"] = None # No hay info sobre las camas, por lo que imprimo None


        try:
            d["Area"] = item.find("span", {"class": "infoSqFt"}).find("b").text
        except:
            d["Area"] = None


        try:
            d["Full Baths"] = item.find("span", {"class": "infoValueFullBath"}).find("b").text
        except:
            d["Full Baths"] = None


        try:
            d["Half Baths"] = item.find("span", {"class": "infoValueHalfBath"}).find("b").text
        except:
            d["Half Baths"] = None
        
        for column_group in item.find_all("div", {"class": "columnGroup"}):
            for feature_group, feature_name in zip(column_group.find_all("span", {"class": "featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
                if "Lot Size" in feature_group.text:
                    d["Lot Size"] = feature_name.text


        l.append(d)



https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html
https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=20.html
